In [94]:
import pandas as pd
import re
import json
import csv

In [81]:
full_wards = []

with open("libbys_scraped_data.json", "r") as f:
    df = json.load(f)
    for row in df:
        full_wards.append(row)
        
with open("scraped_data.json", "r") as f:
    df = json.load(f)
    for row in df:
        full_wards.append(row)
        
all_alderpeople = pd.DataFrame(full_wards)

all_alderpeople

,Ward,Alderperson,Start Date,End Date,Party,Notes
0,5,Paul Howard Douglas,1939,1942,None,None
1,5,Leon Despres,1955,1975,None,None
2,5,Leslie Hairston,May 1999,"May 15, 2023",None,None
3,6,Eugene Sawyer,"February 28, 1971","December 2, 1987",None,None
4,6,Freddrenna Lyle,"February 8, 1998","May 15, 2011",None,None
...,...,...,...,...,...,...
388,49th Ward,David Orr,1979,1990,Democratic,
389,49th Ward,Joe Moore,1991,"May 20, 2019",Democratic,
390,49th Ward,Maria Hadden,"May 20, 2019",present,N/A,
391,50th Ward,Bernard Stone,1973,2011,Democratic,


Okay. There are a couple of things we have to do to streamline our data. 
1) ensure there is only the year in the start and end dates
2) ensure there is only the number in the ward
3) filter data so we are only working with people between 2012 and 2023
    - so if end date is none or present
    - if end date is between 2012 and 2023

In [88]:
all_alderpeople["Clean Ward"] =  all_alderpeople["Ward"].str.extract('(\\d+)')
all_alderpeople["Start Year"] =  all_alderpeople["Start Date"].str.extract('(\\d{4})')
all_alderpeople["End Year"] =  all_alderpeople["End Date"].str.extract('(\\d{4})')
all_alderpeople["End Year"] =  pd.to_numeric(all_alderpeople['End Year'])


In [89]:
all_alderpeople

,Ward,Alderperson,Start Date,End Date,Party,Notes,Clean Ward,Start Year,End Year
0,5,Paul Howard Douglas,1939,1942,None,None,5,1939,1942.0
1,5,Leon Despres,1955,1975,None,None,5,1955,1975.0
2,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0
3,6,Eugene Sawyer,"February 28, 1971","December 2, 1987",None,None,6,1971,1987.0
4,6,Freddrenna Lyle,"February 8, 1998","May 15, 2011",None,None,6,1998,2011.0
...,...,...,...,...,...,...,...,...,...
388,49th Ward,David Orr,1979,1990,Democratic,,49,1979,1990.0
389,49th Ward,Joe Moore,1991,"May 20, 2019",Democratic,,49,1991,2019.0
390,49th Ward,Maria Hadden,"May 20, 2019",present,N/A,,49,2019,NaN
391,50th Ward,Bernard Stone,1973,2011,Democratic,,50,1973,2011.0


In [93]:
alders_2012_2023 =  all_alderpeople.loc[(all_alderpeople['End Year'] >= 2012) | (all_alderpeople['End Year'].isna()) ]

alders_2012_2023

,Ward,Alderperson,Start Date,End Date,Party,Notes,Clean Ward,Start Year,End Year
2,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0
5,6,Roderick Sawyer,"May 16, 2011","May 15, 2023",None,None,6,2011,2023.0
7,7,Sandi Jackson,"May 21, 2007","January 15, 2013",None,None,7,2007,2013.0
8,7,Natashia Holmes,"February 13, 2013","May 18, 2015",None,None,7,2013,2015.0
9,7,Gregory Mitchell,"May 18, 2015",present,None,None,7,2015,NaN
...,...,...,...,...,...,...,...,...,...
385,48th Ward,Joe Moore,1991,"May 20, 2019",Democratic,,48,1991,2019.0
386,48th Ward,Maria Hadden,"May 20, 2019",present,N/A,,48,2019,NaN
389,49th Ward,Joe Moore,1991,"May 20, 2019",Democratic,,49,1991,2019.0
390,49th Ward,Maria Hadden,"May 20, 2019",present,N/A,,49,2019,NaN


In [95]:
alders_2012_2023.to_csv('alders_2012_2023.csv')